In [2]:
    import csv
    import sys


    def mkString(xs, sep):
        return sep.join(map(str,xs))

    def assign(var, val):
        globals()[var] = val


    def convert(val):
        try:
            return float(val)
        except:
            return str(val)


    def is_exclude(header, variant, excl_filters):
        for (h,v) in zip(header,variant):
            assign(h, convert(v))

        reasons = [ filter for filter in excl_filters if eval(filter) ] # CAUTION
        is_excluded = len(reasons) > 0

        return (is_excluded, reasons)


    fname = 'AxiomGT1.calls_1000g_merged_filter_table.txt'
    filters_file = 'AxiomGT1.calls_1000g.exclusion_list.txt'

    exclusion_list_file = open('AxiomGT1.calls_1000g_merged_filter_table.txt', 'w')
    filtered_file = open('AxiomGT1.calls_1000g_merged_filter_table.txt', 'w')

    # example: excl_filters = ['cr < 0.9', 'hwe < 10e-6', 'cr < 0.95 if hwe < 10e-5 else cr < 0.98']
    excl_filters = list(map(str.strip, open(filters_file, "r").readlines()))

    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = '\t')
        header = next(csv_reader)

        new_header = [header[0], 'excludable', 'exclusion_reason'] + header[1:]
        filtered_file.write(mkString(new_header, '\t') + '\n')

        for variant in csv_reader:
            (is_excluded, reasons) = is_exclude(header, variant, excl_filters)

            fmt_reasons = '"'+mkString(reasons, '; ')+'"' if is_excluded else 'NA'
            new_row = [variant[0], is_excluded, fmt_reasons] + variant[1:]
            filtered_file.write(mkString(new_row, '\t') + '\n')

            if is_excluded:
                exclusion_list_file.write(variant[0] + '\n')

    exclusion_list_file.close()

StopIteration: 